In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks
import tensorflow as tf

In [2]:
datagen = keras.preprocessing.image.ImageDataGenerator(
                rescale=1./255)

train_path = os.path.join(os.pardir, os.pardir, 'data/train')
val_path = os.path.join(os.pardir, os.pardir, 'data/val')
test_path = os.path.join(os.pardir, os.pardir, 'data/test')

In [ ]:
image_size = 32
batch_size = 32

train_generator = datagen.flow_from_directory(
                train_path,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')

test_generator = datagen.flow_from_directory(
                test_path,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')

## Test known model to ensure functionality

In [ ]:
IMG_SHAPE = (image_size, image_size, 3)

base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [ ]:
base_model.trainable = False

In [ ]:
model1 = tf.keras.Sequential([
  base_model,
  keras.layers.GlobalAveragePooling2D(),
  keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model1.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
epochs = 10
steps_per_epoch = train_generator.n // batch_size
validation_steps = test_generator.n // batch_size

history = model1.fit_generator(train_generator,
                              steps_per_epoch = steps_per_epoch,
                              epochs=epochs,
                              workers=4,
                              validation_data=test_generator,
                              validation_steps=validation_steps)

## Self-Made Model

In [ ]:
# create some convolutional layers, along with basic dropout for regularization 

model2 = tf.keras.Sequential()

model2.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model2.add(layers.MaxPooling2D((2, 2)))
model2.add(layers.Dropout(.5))
model2.add(layers.Conv2D(64, (3, 3), activation='relu'))
model2.add(layers.MaxPooling2D((2, 2)))
model2.add(layers.Dropout(.3))
model2.add(layers.Conv2D(64, (3, 3), activation='relu'))

model2.add(layers.Flatten())
model2.add(layers.Dense(64, activation='relu'))
model2.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model2.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model2.fit_generator(train_generator,
                     steps_per_epoch = steps_per_epoch,
                     epochs=epochs,
                     workers=4,
                     validation_data=test_generator,
                     validation_steps=validation_steps)

## Resizing images to 160x160


In [ ]:
image_size = 160
batch_size = 32


train_generator160 = datagen.flow_from_directory(
                train_path,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')

test_generator160 = datagen.flow_from_directory(
                test_path,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')


In [ ]:
# create some convolutional layers, along with basic dropout for regularization, 

model3 = tf.keras.Sequential()

model3.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(160, 160, 3)))
model3.add(layers.MaxPooling2D((2, 2)))
model3.add(layers.Dropout(.5))
model3.add(layers.Conv2D(64, (3, 3), activation='relu'))
model3.add(layers.MaxPooling2D((2, 2)))
model3.add(layers.Dropout(.3))
model3.add(layers.Conv2D(64, (3, 3), activation='relu'))

model3.add(layers.Flatten())
model3.add(layers.Dense(64, activation='relu'))
model3.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
epochs = 10
steps_per_epoch160 = train_generator160.n // batch_size
validation_steps160 = test_generator160.n // batch_size

model3.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model3.fit_generator(train_generator160,
                     steps_per_epoch = steps_per_epoch160,
                     epochs=epochs,
                     workers=4,
                     validation_data=test_generator160,
                     validation_steps=validation_steps160)

analysis: variant accuracy indicated dropout may be too high early in network, unclear if issue is overfit or dropout coincidence

solve: remove dropout entirely, then add backwards (from output layer to input layer), until stable and accurate

## Removing Regularization


In [ ]:
image_size = 160
batch_size = 32

train_generator160 = datagen.flow_from_directory(
                train_path,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')

test_generator160 = datagen.flow_from_directory(
                test_path,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')



In [ ]:
# create some convolutional layers, along with basic dropout for regularization, 

model4 = tf.keras.Sequential()

model4.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(160, 160, 3)))
model4.add(layers.MaxPooling2D((2, 2)))
model4.add(layers.Conv2D(64, (3, 3), activation='relu'))
model4.add(layers.MaxPooling2D((2, 2)))
model4.add(layers.Conv2D(64, (3, 3), activation='relu'))

model4.add(layers.Flatten())
model4.add(layers.Dense(64, activation='relu'))
model4.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
epochs = 10
steps_per_epoch160 = train_generator160.n // batch_size
validation_steps160 = test_generator160.n // batch_size

model4.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model4.fit_generator(train_generator160,
                     steps_per_epoch = steps_per_epoch160,
                     epochs=epochs,
                     workers=4,
                     validation_data=test_generator160,
                     validation_steps=validation_steps160)

##  increasing batch size

In [ ]:
image_size = 160
batch_size64 = 64

train_generator160 = datagen.flow_from_directory(
                train_path,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size64,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')

test_generator160 = datagen.flow_from_directory(
                test_path,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size64,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')


In [ ]:
model5 = tf.keras.Sequential()

model5.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(160, 160, 3)))
model5.add(layers.MaxPooling2D((2, 2)))

model5.add(layers.Conv2D(64, (3, 3), activation='relu'))
model5.add(layers.MaxPooling2D((2, 2)))

model5.add(layers.Conv2D(64, (3, 3), activation='relu'))
model5.add(layers.MaxPooling2D((2, 2)))

model5.add(layers.Flatten())
model5.add(layers.Dense(64, activation='relu'))
model5.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
epochs = 10
steps_per_epoch160 = train_generator160.n // batch_size64
validation_steps160 = test_generator160.n // batch_size64

model5.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model5.fit_generator(train_generator160,
                     steps_per_epoch = steps_per_epoch160,
                     epochs=epochs,
                     workers=4,
                     validation_data=test_generator160,
                     validation_steps=validation_steps160)

analysis:  batch size 64 performed slightly better, but lack of regularization hurt the overall performance

solve: add batch normalization to each layer, and view results

## regularization: batch normalization w/ batch size 64

In [ ]:
image_size = 160
batch_size64 = 64

train_generator160 = datagen.flow_from_directory(
                train_path,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size64,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')

test_generator160 = datagen.flow_from_directory(
                test_path,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size64,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')



In [ ]:
model6 = tf.keras.Sequential()

model6.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(160, 160, 3)))
model6.add(layers.MaxPooling2D((2, 2)))

model6.add(layers.Conv2D(64, (3, 3), activation='relu'))
model6.add(layers.BatchNormalization())
model6.add(layers.MaxPooling2D((2, 2)))

model6.add(layers.Conv2D(64, (3, 3), activation='relu'))
model6.add(layers.BatchNormalization())
model6.add(layers.MaxPooling2D((2, 2)))

model6.add(layers.Flatten())
model6.add(layers.Dense(64, activation='relu'))
model6.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
epochs = 10
steps_per_epoch160 = train_generator160.n // batch_size64
validation_steps160 = test_generator160.n // batch_size64

model6.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model6.fit_generator(train_generator160,
                     steps_per_epoch = steps_per_epoch160,
                     epochs=epochs,
                     workers=4,
                     validation_data=test_generator160,
                     validation_steps=validation_steps160)

## regularization: batch normalization w/ batch size 32

In [3]:
image_size = 160
batch_size = 32

train_generator160 = datagen.flow_from_directory(
                train_path,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')

test_generator160 = datagen.flow_from_directory(
                test_path,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [4]:
model7 = tf.keras.Sequential()

model7.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(160, 160, 3)))
model7.add(layers.MaxPooling2D((2, 2)))

model7.add(layers.Conv2D(64, (3, 3), activation='relu'))
model7.add(layers.BatchNormalization())
model7.add(layers.MaxPooling2D((2, 2)))

model7.add(layers.Conv2D(64, (3, 3), activation='relu'))
model7.add(layers.BatchNormalization())
model7.add(layers.MaxPooling2D((2, 2)))

model7.add(layers.Flatten())
model7.add(layers.Dense(64, activation='relu'))
model7.add(layers.Dense(1, activation='sigmoid'))

W0414 14:26:52.197391 4722408896 deprecation.py:506] From /Users/kyledecember1/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [5]:
epochs = 5
steps_per_epoch160 = train_generator160.n // batch_size
validation_steps160 = test_generator160.n // batch_size

model7.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model7.fit_generator(train_generator160,
                     steps_per_epoch = steps_per_epoch160,
                     epochs=epochs,
                     workers=4,
                     validation_data=test_generator160,
                     validation_steps=validation_steps160)

W0414 14:26:53.204540 4722408896 deprecation.py:323] From /Users/kyledecember1/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/5
163/163 [==============================] - 266s 2s/step - loss: 0.2270 - acc: 0.9304 - val_loss: 0.9777 - val_acc: 0.6217
Epoch 2/5
163/163 [==============================] - 229s 1s/step - loss: 0.0756 - acc: 0.9751 - val_loss: 1.2767 - val_acc: 0.6234
Epoch 3/5
163/163 [==============================] - 226s 1s/step - loss: 0.0453 - acc: 0.9837 - val_loss: 1.1286 - val_acc: 0.6546
Epoch 4/5
163/163 [==============================] - 233s 1s/step - loss: 0.0283 - acc: 0.9898 - val_loss: 1.1076 - val_acc: 0.7664
Epoch 5/5
163/163 [==============================] - 207s 1s/step - loss: 0.0156 - acc: 0.9937 - val_loss: 3.9594 - val_acc: 0.6826


## adding dropout layers / increasing total layers of convolution

In [6]:
image_size = 160
batch_size = 32

train_generator160 = datagen.flow_from_directory(
                train_path,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')

test_generator160 = datagen.flow_from_directory(
                test_path,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [7]:
model8 = tf.keras.Sequential()

model8.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(160, 160, 3)))
model8.add(layers.MaxPooling2D((2, 2)))

model8.add(layers.Conv2D(64, (3, 3), activation='relu'))
model8.add(layers.BatchNormalization())
model8.add(layers.MaxPooling2D((2, 2)))

model8.add(layers.Conv2D(128, (3, 3), activation='relu'))
model8.add(layers.BatchNormalization())
model8.add(layers.MaxPooling2D((2, 2)))

model8.add(layers.Conv2D(256, (3, 3), activation='relu'))
model8.add(layers.BatchNormalization())
model8.add(layers.MaxPooling2D((2, 2)))

model8.add(layers.Flatten())
model8.add(layers.Dense(256, activation='relu'))
model8.add(layers.Dropout(.3))
model8.add(layers.Dense(128, activation='relu'))
model8.add(layers.Dropout(.2))
model8.add(layers.Dense(64, activation='relu'))
model8.add(layers.Dropout(.1))

model8.add(layers.Dense(1, activation='sigmoid'))

In [8]:
epochs = 5
steps_per_epoch160 = train_generator160.n // batch_size
validation_steps160 = test_generator160.n // batch_size

model8.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model8.fit_generator(train_generator160,
                     steps_per_epoch = steps_per_epoch160,
                     epochs=epochs,
                     workers=4,
                     validation_data=test_generator160,
                     validation_steps=validation_steps160)

Epoch 1/5
163/163 [==============================] - 281s 2s/step - loss: 0.2128 - acc: 0.9231 - val_loss: 2.2559 - val_acc: 0.6266
Epoch 2/5
163/163 [==============================] - 273s 2s/step - loss: 0.1299 - acc: 0.9555 - val_loss: 4.1423 - val_acc: 0.6266
Epoch 3/5
163/163 [==============================] - 281s 2s/step - loss: 0.0967 - acc: 0.9703 - val_loss: 2.2651 - val_acc: 0.6414
Epoch 4/5
163/163 [==============================] - 317s 2s/step - loss: 0.0745 - acc: 0.9757 - val_loss: 0.3036 - val_acc: 0.8914
Epoch 5/5
163/163 [==============================] - 270s 2s/step - loss: 0.0586 - acc: 0.9835 - val_loss: 5.3896 - val_acc: 0.6447


#### analysis

every model beyond model 3 has only decreased validation accuracy

removing batch normalization from model 8 to investigate performance

## remove batch normalization

In [9]:
image_size = 160
batch_size = 32

train_generator160 = datagen.flow_from_directory(
                train_path,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')

test_generator160 = datagen.flow_from_directory(
                test_path,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [10]:
model9 = tf.keras.Sequential()

model9.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(160, 160, 3)))
model9.add(layers.MaxPooling2D((2, 2)))

model9.add(layers.Conv2D(64, (3, 3), activation='relu'))
model9.add(layers.MaxPooling2D((2, 2)))

model9.add(layers.Conv2D(128, (3, 3), activation='relu'))
model9.add(layers.MaxPooling2D((2, 2)))

model9.add(layers.Conv2D(256, (3, 3), activation='relu'))
model9.add(layers.MaxPooling2D((2, 2)))

model9.add(layers.Flatten())
model9.add(layers.Dense(256, activation='relu'))
model9.add(layers.Dropout(.3))
model9.add(layers.Dense(128, activation='relu'))
model9.add(layers.Dropout(.2))
model9.add(layers.Dense(64, activation='relu'))
model9.add(layers.Dropout(.1))

model9.add(layers.Dense(1, activation='sigmoid'))

In [11]:
epochs = 5
steps_per_epoch160 = train_generator160.n // batch_size
validation_steps160 = test_generator160.n // batch_size

model9.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model9.fit_generator(train_generator160,
                     steps_per_epoch = steps_per_epoch160,
                     epochs=epochs,
                     workers=4,
                     validation_data=test_generator160,
                     validation_steps=validation_steps160)

Epoch 1/5
163/163 [==============================] - 201s 1s/step - loss: 0.4682 - acc: 0.7960 - val_loss: 0.3398 - val_acc: 0.8651
Epoch 2/5
163/163 [==============================] - 199s 1s/step - loss: 0.1990 - acc: 0.9224 - val_loss: 0.6866 - val_acc: 0.7763
Epoch 3/5
163/163 [==============================] - 223s 1s/step - loss: 0.1469 - acc: 0.9461 - val_loss: 0.6164 - val_acc: 0.7763
Epoch 4/5
163/163 [==============================] - 207s 1s/step - loss: 0.1094 - acc: 0.9601 - val_loss: 0.6047 - val_acc: 0.8224
Epoch 5/5
163/163 [==============================] - 208s 1s/step - loss: 0.0979 - acc: 0.9628 - val_loss: 0.8417 - val_acc: 0.7862


#### analysis

improved performance without Batch Normalization


## adding more dropout

In [14]:
image_size = 160
batch_size = 32

train_generator160 = datagen.flow_from_directory(
                train_path,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')

test_generator160 = datagen.flow_from_directory(
                test_path,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [15]:
model10 = tf.keras.Sequential()

model10.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(160, 160, 3)))
model10.add(layers.MaxPooling2D((2, 2)))

model10.add(layers.Conv2D(64, (3, 3), activation='relu'))
model10.add(layers.MaxPooling2D((2, 2)))
model10.add(layers.Dropout(.2))

model10.add(layers.Conv2D(128, (3, 3), activation='relu'))
model10.add(layers.MaxPooling2D((2, 2)))
model10.add(layers.Dropout(.2))

model10.add(layers.Conv2D(256, (3, 3), activation='relu'))
model10.add(layers.MaxPooling2D((2, 2)))
model10.add(layers.Dropout(.2))

model10.add(layers.Flatten())
model10.add(layers.Dense(256, activation='relu'))
model10.add(layers.Dropout(.3))
model10.add(layers.Dense(128, activation='relu'))
model10.add(layers.Dropout(.2))
model10.add(layers.Dense(64, activation='relu'))
model10.add(layers.Dropout(.1))

model10.add(layers.Dense(1, activation='sigmoid'))

In [16]:
epochs = 5
steps_per_epoch160 = train_generator160.n // batch_size
validation_steps160 = test_generator160.n // batch_size

model10.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model10.fit_generator(train_generator160,
                     steps_per_epoch = steps_per_epoch160,
                     epochs=epochs,
                     workers=4,
                     validation_data=test_generator160,
                     validation_steps=validation_steps160)

Epoch 1/5
163/163 [==============================] - 288s 2s/step - loss: 0.5253 - acc: 0.7584 - val_loss: 0.4900 - val_acc: 0.8421
Epoch 2/5
163/163 [==============================] - 229s 1s/step - loss: 0.2897 - acc: 0.8806 - val_loss: 0.3709 - val_acc: 0.8553
Epoch 3/5
163/163 [==============================] - 227s 1s/step - loss: 0.2027 - acc: 0.9179 - val_loss: 0.5105 - val_acc: 0.7763
Epoch 4/5
163/163 [==============================] - 233s 1s/step - loss: 0.1551 - acc: 0.9406 - val_loss: 0.3815 - val_acc: 0.8372
Epoch 5/5
163/163 [==============================] - 225s 1s/step - loss: 0.1346 - acc: 0.9469 - val_loss: 0.4697 - val_acc: 0.7829


#### analysis

still overfitting to training data, increasing dropout rates

In [17]:
image_size = 160
batch_size = 32

train_generator160 = datagen.flow_from_directory(
                train_path,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')

test_generator160 = datagen.flow_from_directory(
                test_path,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [18]:
model11 = tf.keras.Sequential()

model11.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(160, 160, 3)))
model11.add(layers.MaxPooling2D((2, 2)))

model11.add(layers.Conv2D(64, (3, 3), activation='relu'))
model11.add(layers.MaxPooling2D((2, 2)))
model11.add(layers.Dropout(.3))

model11.add(layers.Conv2D(128, (3, 3), activation='relu'))
model11.add(layers.MaxPooling2D((2, 2)))
model11.add(layers.Dropout(.4))

model11.add(layers.Conv2D(256, (3, 3), activation='relu'))
model11.add(layers.MaxPooling2D((2, 2)))
model11.add(layers.Dropout(.6))

model11.add(layers.Flatten())
model11.add(layers.Dense(256, activation='relu'))
model11.add(layers.Dropout(.6))
model11.add(layers.Dense(128, activation='relu'))
model11.add(layers.Dropout(.4))
model11.add(layers.Dense(64, activation='relu'))
model11.add(layers.Dropout(.2))
model11.add(layers.Dense(1, activation='sigmoid'))

W0414 16:09:28.746411 4722408896 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W0414 16:09:28.870649 4722408896 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


In [19]:
epochs = 5
steps_per_epoch160 = train_generator160.n // batch_size
validation_steps160 = test_generator160.n // batch_size

model11.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model11.fit_generator(train_generator160,
                     steps_per_epoch = steps_per_epoch160,
                     epochs=epochs,
                     workers=4,
                     validation_data=test_generator160,
                     validation_steps=validation_steps160)

Epoch 1/5
163/163 [==============================] - 234s 1s/step - loss: 0.6007 - acc: 0.7239 - val_loss: 0.6669 - val_acc: 0.6266
Epoch 2/5
163/163 [==============================] - 219s 1s/step - loss: 0.5311 - acc: 0.7429 - val_loss: 0.5642 - val_acc: 0.6266
Epoch 3/5
163/163 [==============================] - 218s 1s/step - loss: 0.3621 - acc: 0.8129 - val_loss: 0.4023 - val_acc: 0.8602
Epoch 4/5
163/163 [==============================] - 220s 1s/step - loss: 0.2452 - acc: 0.9028 - val_loss: 0.4912 - val_acc: 0.8174
Epoch 5/5
163/163 [==============================] - 232s 1s/step - loss: 0.2082 - acc: 0.9183 - val_loss: 0.3527 - val_acc: 0.8520
